In [ ]:
# All brands has been explored and preproceesed and saved as csv

  # https://www.kaggle.com/enginsights/vw-preprocessing-and-exploration
  # https://www.kaggle.com/enginsights/vauxhall-preprocessing-and-exploration
  # https://www.kaggle.com/enginsights/bmw-preprocessing-and-exploration
  # https://www.kaggle.com/enginsights/audi-preprocessing-and-exploration
  # https://www.kaggle.com/enginsights/mercedes-preprocessing-and-exploration
  # https://www.kaggle.com/enginsights/skoda-preprocessing-and-exploration
  # https://www.kaggle.com/enginsights/toyota-preprocessing-and-exploration
  # https://www.kaggle.com/enginsights/ford-preprocessing-and-exploration
  # https://www.kaggle.com/enginsights/hyundai-preprocessing-and-exploration

# All CSVs of processed data of all brands imported to separate notebook where a regression model is applied

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Packages

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 11000)
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

## Loading Pre-Processed Data

In [ ]:
audi_final = pd.read_csv('/kaggle/input/100k-uk-used-cars-processed-files/audi_milefinal.csv')
bmw_final = pd.read_csv('/kaggle/input/100k-uk-used-cars-processed-files/bmw_milefinal.csv')
ford_final = pd.read_csv('/kaggle/input/100k-uk-used-cars-processed-files/ford_milefinal.csv')
hyundi_final = pd.read_csv('/kaggle/input/100k-uk-used-cars-processed-files/hyundi_milefinal.csv')
merc_final = pd.read_csv('/kaggle/input/100k-uk-used-cars-processed-files/merc_milefinal.csv')
skoda_final = pd.read_csv('/kaggle/input/100k-uk-used-cars-processed-files/skoda_milefinal.csv')
toyota_final = pd.read_csv('/kaggle/input/100k-uk-used-cars-processed-files/toyota_milefinal.csv')
vauxhall_final = pd.read_csv('/kaggle/input/100k-uk-used-cars-processed-files/vauxhall_milefinal.csv')
vw_final = pd.read_csv('/kaggle/input/100k-uk-used-cars-processed-files/vw_milefinal.csv')

## Counting Brands

In [ ]:
audi_count=len(audi_final['model'])
bmw_count=len(bmw_final['model'])
ford_count=len(ford_final['model'])
hyundi_count=len(hyundi_final['model'])
merc_count=len(merc_final['model'])
toyota_count=len(toyota_final['model'])
vauxhall_count=len(vauxhall_final['model'])
vw_count=len(vw_final['model'])

In [ ]:
brands=['audi','bmw','ford','hyundi','merc','toyota','vauxhall','vw']
counts=[audi_count,bmw_count,ford_count,hyundi_count,merc_count,toyota_count,vauxhall_count,vw_count]

In [ ]:
brands_counts=pd.DataFrame()

In [ ]:
brands_counts['Brand']=brands
brands_counts['Count']=counts

In [ ]:
brands_counts.sort_values(by=['Count'], inplace=True)
brands_counts.plot(kind='bar',x='Brand',y='Count')

## Union Dataframes

In [ ]:
uk_cars_final=pd.concat([audi_final,bmw_final,ford_final,hyundi_final,merc_final,skoda_final,toyota_final,vauxhall_final,
                         vw_final])

In [ ]:
uk_cars=uk_cars_final.reset_index(drop=True)

## Further Data Exploration

In [ ]:
uk_cars.describe(include='all')

In [ ]:
fig, axes = plt.subplots(3,figsize=(10,10))
sns.distplot(uk_cars['price'],ax=axes[0])
sns.distplot(uk_cars['engineSize'],ax=axes[1])
sns.distplot(uk_cars['mileage'],ax=axes[2])

## Further Data Processing

In [ ]:
uk_car_cleaning=uk_cars.copy()

### FuelType

In [ ]:
vis_ftyp = uk_car_cleaning.groupby(['fuelType']).count()['price'].sort_values(ascending=False)
vis_ftyp.plot(kind='bar',title = 'Count')

In [ ]:
uk_car_cleaning = uk_car_cleaning[uk_car_cleaning.fuelType != 'Hybrid']
uk_car_cleaning = uk_car_cleaning[uk_car_cleaning.fuelType != 'Other']

### Transmission

In [ ]:
vis_ftyp = uk_car_cleaning.groupby(['transmission']).count()['price'].sort_values(ascending=False)
vis_ftyp.plot(kind='bar',title = 'Count')

In [ ]:
uk_car_cleaning = uk_car_cleaning[uk_car_cleaning.transmission != 'Other']

### Model

In [ ]:
len(uk_car_cleaning['model'].unique())

In [ ]:
vis_cars_model = uk_car_cleaning.groupby(['model']).count()['price'].sort_values(ascending=False)

In [ ]:
vis_cars_model.plot(kind='bar',figsize=(20,5),rot=90)

In [ ]:
brand_model_count=pd.DataFrame
brand_model_count=brand_model_count({'counts' : uk_car_cleaning.groupby( [ "model"] ).size()}).reset_index()
brand_model_count.drop(brand_model_count[brand_model_count.counts >= 2].index, inplace=True)
a=brand_model_count['model'].tolist()

In [ ]:
for i in range (0,len(a)):
    uk_car_cleaning = uk_car_cleaning[uk_car_cleaning.model !=a[i]]

In [ ]:
vis_cars_model = uk_car_cleaning.groupby(['model']).count()['price'].sort_values(ascending=False)

In [ ]:
vis_cars_model = uk_car_cleaning.groupby(['model']).min()['price'].sort_values(ascending=False)

In [ ]:
uk_car_cleaning.boxplot(column=['price'], by=['model'],figsize=(30,5),rot=90)

In [ ]:
## As shown prices are not exceeding 80,000. However, there are few cars exceeding this value but as an outliers.
## I will drop prices above 80,000

In [ ]:
uk_car_cleaning = uk_car_cleaning[uk_car_cleaning.price <80000]

In [ ]:
len(uk_car_cleaning['model'].unique())

In [ ]:
uk_cars_cleaned=uk_car_cleaning.copy()

## Linear Regression Assumptions

## Linearity

In [ ]:
f, (ax2, ax3) = plt.subplots(1, 2, sharey=True, figsize =(20,5))
ax2.scatter(uk_cars_cleaned['mileage'],uk_cars_cleaned['price'])
ax2.set_title('Price and mile')
ax3.scatter(uk_cars_cleaned['engineSize'],uk_cars_cleaned['price'])
ax3.set_title('Price and engineSize')
plt.show()

In [ ]:
log_price = np.log(uk_cars_cleaned['price'])
uk_cars_cleaned['log_price'] = log_price

In [ ]:
f, (ax2, ax3) = plt.subplots(1, 2, sharey=True, figsize =(20,5))
ax2.scatter(uk_cars_cleaned['mileage'],uk_cars_cleaned['log_price'])
ax2.set_title('log_price and mile')
ax3.scatter(uk_cars_cleaned['engineSize'],uk_cars_cleaned['log_price'])
ax3.set_title('log_price and engineSize')
plt.show()

## Multicollinearity

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
variables = uk_cars_cleaned[['mileage','engineSize']]
vif = pd.DataFrame()
vif["VIF"] = [variance_inflation_factor(variables.values, i) for i in range(variables.shape[1])]
vif["features"] = variables.columns

In [ ]:
vif

# Linear Regression Processed Data

In [ ]:
uk_cars_LRdata=uk_cars_cleaned.copy()

## Droping Price Field

In [ ]:
uk_cars_LRdata = uk_cars_LRdata.drop(['price'],axis=1)

## Rearrange a bit

In [ ]:
uk_cars_LRdata.columns.values

In [ ]:
cols = ['log_price', 'model', 'transmission', 'fuelType', 'engineSize', 'mileage',]

In [ ]:
uk_cars_LRdata_arr = uk_cars_LRdata[cols]

In [ ]:
uk_cars_LRdata_arr.head()

In [ ]:
df_numeric_cols=uk_cars_LRdata_arr.select_dtypes(include='number')
col_num=len(df_numeric_cols.columns)-1

## Creating Dummies

In [ ]:
uk_cars_LRdata_arr['engineSize'] = uk_cars_LRdata_arr['engineSize'].astype(str)

In [ ]:
uk_cars_LRdata_dummies = pd.get_dummies(uk_cars_LRdata_arr, drop_first=True)

In [ ]:
uk_cars_LRdata_dummies.head()

## Shuffling Data

In [ ]:
uk_cars_shuffled = uk_cars_LRdata_dummies.sample(frac = 1)

In [ ]:
uk_cars_shuffled.describe(include='all')

# Linear Regression Model

## Packages

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

## Declare Inputs and Targets

In [ ]:
targets = uk_cars_shuffled['log_price']
unscaled_inputs = uk_cars_shuffled.drop(['log_price'],axis=1)

## Scaling

In [ ]:
scaler = StandardScaler()
# import the libraries needed to create the Custom Scaler
# note that all of them are a part of the sklearn package
# moreover, one of them is actually the StandardScaler module, 
# so you can imagine that the Custom Scaler is build on it

from sklearn.base import BaseEstimator, TransformerMixin

# create the Custom Scaler class

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    # init or what information we need to declare a CustomScaler object
    # and what is calculated/declared as we do
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        
        # scaler is nothing but a Standard Scaler object
        self.scaler = StandardScaler(copy,with_mean,with_std)
        # with some columns 'twist'
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    
    # the fit method, which, again based on StandardScale
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    # the transform method which does the actual scaling

    def transform(self, X, y=None, copy=None):
        
        # record the initial order of the columns
        init_col_order = X.columns
        
        # scale all features that you chose when creating the instance of the class
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        
        # declare a variable containing all information that was not scaled
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        
        # return a data frame which contains all scaled features and all 'not scaled' features
        # use the original order (that you recorded in the beginning)
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [ ]:
cols=unscaled_inputs.columns.values

In [ ]:
# select the columns to omit
columns_to_omit = cols[col_num:]
# create the columns to scale, based on the columns to omit
# use list comprehension to iterate over the list
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [ ]:
# fit the data (calculate mean and standard deviation); they are automatically stored inside the object 
scaler.fit(unscaled_inputs)

# standardizes the data, using the transform method 
# in the last line, we fitted the data - in other words
# we found the internal parameters of a model that will be used to transform data. 
# transforming applies these parameters to our data
# note that when you get new data, you can just call 'scaler' again and transform it in the same way as now
scaled_inputs = scaler.transform(unscaled_inputs)

# check the shape of the inputs
scaled_inputs.shape

## Train - Test Split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, test_size=0.2, random_state=365)

## Create the regression

In [ ]:
reg = LinearRegression()
reg.fit(x_train,y_train)

In [ ]:
y_hat = reg.predict(x_train)

In [ ]:
plt.scatter(y_train, y_hat,alpha=0.2)
plt.xlabel('Targets',size=18)
plt.ylabel('Predictions',size=18)
plt.show()

In [ ]:
sns.distplot(y_train - y_hat)
plt.title("Residuals PDF", size=18)

In [ ]:
reg_score=reg.score(x_train,y_train)*100
reg_score

## Finding the weights and bias

In [ ]:
reg.intercept_

In [ ]:
reg_summary = pd.DataFrame(unscaled_inputs.columns.values, columns=['Features'])
reg_summary['Weights'] = reg.coef_
reg_summary

In [ ]:
uk_cars_LRdata_arr['model'].unique()

## Testing

In [ ]:
y_hat_test = reg.predict(x_test)

In [ ]:
plt.scatter(y_test, y_hat_test, alpha=0.2)
plt.xlabel('Targets (y_test)',size=18)
plt.ylabel('Predictions (y_hat_test)',size=18)
plt.show()

In [ ]:
df_pf = pd.DataFrame(np.exp(y_hat_test), columns=['Prediction'])

In [ ]:
y_test = y_test.reset_index(drop=True)

In [ ]:
df_pf['Target'] = np.exp(y_test)

In [ ]:
df_pf['Residual'] = df_pf['Target'] - df_pf['Prediction']

In [ ]:
df_pf['Difference%'] = np.absolute(df_pf['Residual']/df_pf['Target']*100)

In [ ]:
df_pf.describe()

In [ ]:
pd.options.display.max_rows = 100
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df_pf.sort_values(by=['Difference%'])

In [ ]:
df_pf.plot(kind='scatter',x='Target',y='Difference%',alpha=0.1)